In [1]:
from numpy import sort
import numpy as np
import pandas as pd
import numpy as np
from numpy import max
from numpy.linalg import norm
import math
import os
import operator

In [2]:
def process_file(f):
    for line in f:
        columns = line.split()
        if columns:
            yield columns

def read_transforms(filename):
    transforms = []
    with open(filename, 'r') as transforms_file:
        for data in process_file(transforms_file):
            transforms.append(list(map(float, data[1:])))
    return transforms
    
def read_model(filename):
    vertices = []
    faces = []
    with open(filename, 'r') as model_file:
        for data in process_file(model_file):
            if data[0] == 'v':
                vertices.append(list(map(float, data[1:])))
            elif data[0] == 'f':
                faces.append(list(map(int, data[1:])))
    return np.array(vertices, np.float), np.array(faces, np.int)

def read_models(filename):
    models_vertecies = []
    models_faces = []
    transforms = []
    with open(filename, 'r') as model_file:
        n_models = int(model_file.readline())
        for data in process_file(model_file):
            vertecies, faces = read_model(data[0])
            transform = list(map(float, data[1:]))
            models_vertecies.append(vertecies)
            models_faces.append(faces)
            transforms.append(transform)
    return (n_models,
            np.array(models_vertecies, np.float),
            np.array(models_faces, np.int),
            transforms)
    
def read_input_files(space_file, result_file):
    n_models, models_vertecies, models_faces, initial_transforms = read_models(space_file)
    result = np.reshape(read_transforms(result_file), (-1, n_models, 7)) 
    transforms = np.concatenate(([initial_transforms], result))
    return models_vertecies, models_faces, transforms


In [3]:
R = 300
max_step = 50
MAX_ANGLE = 5 / 180 * 3.1415926 - 0.00001

In [4]:
def rotate_at_last_transform(rockets):
    for i in range((np.array(rockets)).shape[1]):
        if(rockets[-1][i][1] > MAX_ANGLE):
            rockets[-1][i][1] = rockets[-1][i][1] - MAX_ANGLE
        elif(rockets[-1][i][1] < (0 - MAX_ANGLE)):
            rockets[-1][i][1] = rockets[-1][i][1] + MAX_ANGLE
        else:
            rockets[-1][i][1] = 0
    return rockets

In [5]:
def homotopic(rockets):
    min_dist = 2 * R
    for i in range(rockets.shape[1]):
        for j in range(i + 1, rockets.shape[1]):
            v = rockets[-1][i][5:] - rockets[-1][j][5:]
            curd = np.linalg.norm(v)
            if(curd < min_dist):
                min_dist = curd
    coef = 2 * R / (min_dist + 1e-3)
    tensor = rockets[-1]
    vectors = list()
    if coef <= 1:
        return rockets
    vectors.append(np.array([0, 0, 0]))
    for i in range(1, rockets.shape[1]):
        vectors.append(coef * (tensor[i][5:] - tensor[0][5:]))
    max_vector = max(norm(vectors))
    num_steps = math.ceil(max_vector/max_step)
    for i in range(num_steps):
        steps = [[0] * rockets.shape[2] for m in range(rockets.shape[1])]
        for j in range(rockets.shape[1]):
            steps[j][:5]   = tensor[j][:5]
            steps[j][5:]   = tensor[j][5:] + np.multiply(vectors[j],1/float(num_steps))
        rockets = np.append(rockets, sort_by_num([steps]), axis=0)
        tensor = sort_by_val(rockets[-1])
    return rockets

In [6]:
def sort_by_val(state):
    df = pd.DataFrame(state)
    df.sort_values(by=[7], inplace =True)
    return (df.values)

In [7]:
def sort_by_num(state):
    luk = np.zeros((1, len(state[0]), 8))
    i = 0
    j = 0
    while i < len(state[0]):
        if state[0][j][0] == i:
            luk[0][i] = state[0][j]
            i += 1
            j = 0
        else: j += 1
    return (luk)

In [8]:
def rank_by_level(rockets):
    last_state = rockets[-1]
    last_state_sort = sort_by_val(last_state)
    rockets[-1] = last_state_sort
    rockets = homotopic(rockets)
    tensor = sort_by_val(rockets[-1])
    while(tensor[0][7] < R):
        for i in range(tensor.shape[0]):
            tensor[i][7] += 50
        rockets = np.append(rockets, sort_by_num([tensor]), axis=0)
        tensor = sort_by_val(rockets[-1])
    for i in range(tensor.shape[0] - 1):
        while((tensor[i + 1][7] - tensor[i][7]) < 2 * R):
            for j in range(i + 1, tensor.shape[0]):
                tensor[j][7] += 50
            rockets = np.append(rockets, sort_by_num([tensor]), axis=0) 
            rockets = rotate_at_last_transform(rockets)
            tensor = sort_by_val(rockets[-1])
    return rockets

In [9]:
def read_from_file(filename):
    dict_ = {} 
    file = open(filename, 'r')
    n = int (file.readline())
    i = 0
    rockets = np.zeros((n, 7))
    for line in file: 
        dict_[i] = line[9]
        rockets[i] = np.array(line[15:].split()).astype(np.float)
        i += 1
    rockets = np.append(np.arange(n).reshape(-1, 1) , rockets, axis=1)
    rockets =  np.expand_dims(rockets, axis=0)
    return rockets, dict_

In [10]:
def write_in_file(filename, rockets):
    file = open(filename, "w") 
    for i in range(rockets.shape[0]):
        np.savetxt(file,rockets[i],  fmt='%10.5f')
    file.close()  

In [11]:
Radius = 170
def R_(j, d):
    d_s = sorted(d.items(), key=operator.itemgetter(1))
    if (int(d_s[j][1]) == 1):
        return Radius * 1.0
    if (int(d_s[j][1]) == 2):
        return Radius * 0.9
    if (int(d_s[j][1]) == 3):
        return Radius * 0.8
    if (int(d_s[j][1]) == 4):
        return Radius * 0.7

In [12]:
def sort_by_r(state, d):
    def f(row):
        return d[row[0]]
    df = pd.DataFrame(state)
    df['sor'] = 0
    df['sor'] = df.apply(f, axis=1)
    df.sort_values(by=['sor'], inplace =True)
    df.drop(['sor'], axis=1, inplace=True)
    return (df.values)

In [13]:
def levels(rockets, d):
    tensor = sort_by_r(rockets[-1], d)
    # find proper line
    y_opt  = 0
    x_opt  = 0
    x_opt_ = 0
    for i in range(rockets.shape[1]):
        y_opt += tensor[i][6]
    y_opt /= tensor.shape[1]

    # find proper initial x for the biggest ball
    for i in range(1, tensor.shape[0]):
        for j in range(1, i):
            x_opt += 2 * R_(j, d)
        x_opt += R_(0, d) + R_(i, d) - tensor[i][5]
    x_opt = (tensor[0][5] - x_opt) / tensor.shape[0]

    for i in range(1, tensor.shape[0]):
        for j in range(1, i):
            x_opt -= 2 * R_(j, d)
        x_opt += -R_(0,d) - R_(i,d) - tensor[i][5]
    x_opt = (tensor[0][5] - x_opt) / tensor.shape[0]
    
    flag=0
    if x_opt > x_opt_:
        flag = 1
        x_opt = x_opt_

    # calculate destinations and directions
    destination = list()
    vectors = list()
    shift = 0
    for i in range(tensor.shape[0]):
        for j in range(1, i):
            shift += 2*R_(j,d)
        if not i == 0:
            shift += R_(0,d)+R_(i,d)
        #destination[i] = [x_opt+shift, y_opt, rockets[-1][i][-1]]
        if flag==1:
            vectors.append([x_opt+shift, y_opt, tensor[i][-1]] - tensor[i][5:])
        else: 
            vectors.append([x_opt-shift, y_opt, tensor[i][-1]] - tensor[i][5:])
        shift = 0

    # calculate number of steps
    max_vector = max(norm(vectors))
    num_steps = math.ceil(max_vector/max_step)

    # move balls
    for i in range(num_steps):
        steps = [[0] * tensor.shape[1] for m in range(tensor.shape[0])]
        for j in range(tensor.shape[0]):
            steps[j][:5] = tensor[j][:5]
            steps[j][5:]     = tensor[j][5:] + vectors[j] / num_steps
        
        rockets = np.append(rockets, sort_by_num([steps]), axis=0)
        rockets = rotate_at_last_transform(rockets)
        tensor = sort_by_r(rockets[-1], d)
        
    return rockets

In [14]:
def scale(d, i):
    if int(d[i]) == 1:
        return 1.0
    if int(d[i]) == 2:
        return 0.9
    if int(d[i]) == 3:
        return 0.8
    if int(d[i]) == 4:
        return 0.7

In [15]:
def down(rockets, d):
    vectors =list()
    for i in range(rockets.shape[1]):
        vectors.append([0, 0, scale(d, rockets[-1][i][0]) * 138-rockets[-1][i][7]])
    max_vector = max(norm(vectors))
    num_steps = math.ceil(max_vector/max_step)
    for i in range(num_steps):
        steps = [[0] * rockets.shape[2] for m in range(rockets.shape[1])]
        for j in range(rockets.shape[1]):
            steps[j][:5] = rockets[-1][j][:5]
            steps[j][5:]   = rockets[-1][j][5:] + np.array(vectors[j])  / num_steps
        rockets = np.append(rockets, sort_by_num([steps]), axis=0)
       # rockets = rotate_at_last_transform(rockets)
        
    return rockets

In [16]:
def is_not_vertical(rockets):
    for i in range(rockets.shape[1]):
        if rockets[-1][i][1] != 0:
            return True
    return False

In [17]:
def rotor(rockets):
    while(is_not_vertical(rockets)):
        vectors = rockets[-1]
        rockets = np.append(rockets, sort_by_num([vectors]), axis=0)
        rockets = rotate_at_last_transform(rockets)
    return rockets

In [20]:
for i in range(10):
    rockets, d = read_from_file("./space-00" + str(i) + '.txt')
    rockets = rank_by_level(rockets)
    rockets = levels(rockets, d)
    rockets = rotor(rockets)
    rockets = down(rockets, d)
    rockets[0] = sort_by_num(rockets)
    write_in_file('./our_results/result' + str(i) + '.txt', rockets)

In [19]:
for i in range(10, 51):
    rockets, d = read_from_file("./space-0" + str(i) + '.txt')
    rockets = rank_by_level(rockets)
    rockets = levels(rockets, d)
    rockets = rotor(rockets)
    rockets = down(rockets, d)
    rockets[0] = sort_by_num(rockets)
    write_in_file('./our_results/result' + str(i) + '.txt', rockets)